tfidf로 해보기

In [26]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from konlpy.tag import Twitter
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

In [2]:
def logistic_recall_rate(X, y, weight=None, len_row=6422, rdst=0):
    model = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('clf', LogisticRegression())])
    cv = ShuffleSplit(len_row, random_state=rdst)
    recall_rate = cross_val_score(model, X, y, scoring='recall', cv=cv, fit_params={'clf__sample_weight':weight})
    return recall_rate

### stratified K-fold

In [87]:
unfrozen = np.array(df_unfrozen['text2'])
frozen = np.array(df_frozen['text'])
weight0 = np.append(np.array(np.ones(3211, dtype=int)), np.array(df_frozen['freeze']))
seodam_x = np.append(unfrozen, frozen)
seodam_y = np.append(np.zeros(3211, dtype=int), np.ones(3211, dtype=int))

In [88]:
tagger = Twitter()
def tokenize(doc):
    token_list = []
    for t in tagger.pos(doc, norm=True, stem=True):
        if t[1] != 'Josa' and t[1] != 'Punctuation' and t[1] != 'Determiner' and t[1] != 'URL' :
            token_list.append('/'.join(t))
    return token_list

In [90]:
model = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('clf', LogisticRegression())])

without sample rate

In [91]:
cv = StratifiedKFold(seodam_y, n_folds=10, random_state=0)
for train_index, test_index in cv:
    X_train0 = seodam_x[train_index]
    X_test0 = seodam_x[test_index]
    y_train = seodam_y[train_index]
    
    model.fit(X_train0, y_train)
    result = model.predict(X_test0)
    
    print classification_report(seodam_y[test_index], result)
    print '*' *70

             precision    recall  f1-score   support

          0       0.62      0.72      0.66       322
          1       0.66      0.56      0.61       322

avg / total       0.64      0.64      0.64       644

**********************************************************************
             precision    recall  f1-score   support

          0       0.71      0.72      0.71       321
          1       0.71      0.70      0.71       321

avg / total       0.71      0.71      0.71       642

**********************************************************************
             precision    recall  f1-score   support

          0       0.67      0.75      0.71       321
          1       0.71      0.63      0.67       321

avg / total       0.69      0.69      0.69       642

**********************************************************************
             precision    recall  f1-score   support

          0       0.74      0.79      0.77       321
          1       0.78      0.72   

with sample weight

In [89]:
cv = StratifiedKFold(seodam_y, n_folds=10, random_state=0)
for train_index, test_index in cv:
    X_train0 = seodam_x[train_index]
    X_test0 = seodam_x[test_index]
    y_train = seodam_y[train_index]
    weight = weight0[train_index]
    
    vect = CountVectorizer(tokenizer=tokenize)
    X_train = vect.fit_transform(X_train0)
    X_test = vect.transform(X_test0)
    
    clf = LogisticRegression().fit(X_train, y_train, sample_weight=weight)
    result = clf.predict(X_test)
    
    print classification_report(seodam_y[test_index], result)
    print '*' *50

             precision    recall  f1-score   support

          0       0.63      0.67      0.65       322
          1       0.65      0.61      0.63       322

avg / total       0.64      0.64      0.64       644

**************************************************
             precision    recall  f1-score   support

          0       0.71      0.68      0.69       321
          1       0.69      0.72      0.70       321

avg / total       0.70      0.70      0.70       642

**************************************************
             precision    recall  f1-score   support

          0       0.69      0.72      0.71       321
          1       0.71      0.67      0.69       321

avg / total       0.70      0.70      0.70       642

**************************************************
             precision    recall  f1-score   support

          0       0.75      0.78      0.76       321
          1       0.77      0.74      0.76       321

avg / total       0.76      0.76      0.7